#### import required libraries


In [1]:

import openai  
import my_secrets
import os
import subprocess
from sentence_transformers import SentenceTransformer  
import faiss  
import numpy as np
import chunks

`Using sentence Transformer for embeddings`


In [2]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
dimension = 384
index = faiss.IndexFlatL2(dimension)
embeddings = []
MyChunks = chunks.Mychunks

make embeddings


In [13]:
def embed_and_index_chunks():
    global embeddings
    if embeddings:
        print("Chunks already embedded and indexed.")
        return
    for chunk in MyChunks:
        text = chunk['text']
        embedding = embedding_model.encode(text)
        embeddings.append(embedding)
        index.add(np.array([embedding]).astype("float32"))
    print("all ok - chunks embedded and indexed")


get top-k similar sentences based on query sentence


In [14]:
def retrieve_chunks(query, top_k=10):
    query_embedding = embedding_model.encode(query)
    distances, indices = index.search(
        np.array([query_embedding]).astype("float32"), top_k)
    print("All ok - chunks retrieved")
    return [MyChunks[i] for i in indices[0]]

generate answer based on chunks


In [15]:
def generate_answer(query, retrieved_chunks):
    context = "\n\n".join(
        [f"Page {chunk['page']}: {chunk['text']}" for chunk in retrieved_chunks])
    print("All ok - answer generated")
    return f"Answer the question based on the following textbook content:\n\n{context}\n\nQuestion: {query}\nAnswer:"


`Running the Rag pipeline`


In [16]:
def run_rag_pipeline(pdf_path, query):
    if not MyChunks:
        return "No data available."
    embed_and_index_chunks()
    retrieved_chunks = retrieve_chunks(query,5)
    print("All ok - chunks retrieved")
    return generate_answer(query, retrieved_chunks)

`define openai key`


In [17]:
def load_openai_key():
    return my_secrets.OPEN_AI_SECRET_KEY

In [8]:
def call_openai_chat(prompt,system_msg, model="gpt-3.5-turbo", max_tokens=4000):
    openai.api_key = load_openai_key()
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "system", "content": {system_msg}},
                      {"role": "user", "content": prompt}],
            max_tokens=4000,
            temperature=0.7
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"An error occurred: {e}"

`save latex code to file`


In [9]:
def save_to_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)

`compiling latex code to pdf`


In [10]:
def compile_latex_to_pdf(tex_file):
    try:
        subprocess.run(["pdflatex", tex_file], check=True)
        pdf_file = tex_file.replace('.tex', '.pdf')
        return pdf_file if os.path.exists(pdf_file) else None
    except subprocess.CalledProcessError as e:
        print(f"Error compiling LaTeX: {e}")
        return None

`clean latex code`


In [11]:
def extract_latex_code(raw_response):
    lines = raw_response.split("\n")
    in_code_block = False
    cleaned_lines = []
    for line in lines:
        if line.strip().startswith("```latex"):
            in_code_block = True
            continue
        if line.strip() == "```":
            in_code_block = False
            continue
        if in_code_block:
            cleaned_lines.append(line)
    return "\n".join(cleaned_lines)

`main`


In [18]:
# Cell 12: Main execution - Get user's prompt, process the document, generate LaTeX Beamer code, and compile into PDF

# Step 1: Get the user's query
user_prompt = input("Enter your prompt: ")

# Step 2: Retrieve relevant chunks from the textbook
pdf_path = "Physics 9.pdf"
answer = run_rag_pipeline(pdf_path, user_prompt)
print(answer)


my_prompt = "Read the following text and improve the text:\n\n" + answer

# call openai chat
msg="You are an expert in Physics"
response = call_openai_chat(my_prompt,msg)
print(response)
# # Step 3: Generate LaTeX Beamer code using OpenAI
# beamer_prompt = (
#     f"Create a detailed LaTeX Beamer presentation on the following topic:\n\n"
#     f"{answer}\n\n"
#     "Include equations, bullet points, and TikZ-based diagrams to illustrate concepts. Use only TikZ to draw shapes or vectors "
#     "instead of relying on external image files. Ensure the output is ready-to-compile Beamer code."
# )
# msg="You are an expert in LaTeX and Beamer"
# raw_beamer_code = call_openai_chat(beamer_prompt,msg)

# # Step 4: Extract valid LaTeX code
# beamer_code = extract_latex_code(raw_beamer_code)

# # Step 5: Save LaTeX code to a .tex file
# tex_filename = "response.tex"
# save_to_file(tex_filename, beamer_code)

# # Step 6: Compile the .tex file into a PDF
# pdf_filename = compile_latex_to_pdf(tex_filename)
# if pdf_filename:
#     print(f"PDF generated: {pdf_filename}")
# else:
#     print("Failed to generate PDF.")


Chunks already embedded and indexed.
All ok - chunks retrieved
All ok - chunks retrieved
All ok - answer generated
Answer the question based on the following textbook content:

Page 80:  Newton's second law of motion states that when a net force acts on a body, it produces acceleration in the body in the direction of the net force. The magnitude of this acceleration is directly proportional to the net force acting on it and inversely proportional to its mass. Mathematically, F = ma.  SI unit of force is newton (N). It is defined as the force which produces 2 an acceleration of 1 ms" in a body of mass 1 kg.  Mass of a body is the quantity of matter possessed by it. It is a scalar

Page 107: the Earth. This unit deals with the concepts related to gravitation. 5.1 THE FORCE OF GRAVITATION On the basis of his observations, Newton concluded that the force which causes an apple to fall on the Earth and the force which keeps the moon in its orbit are of the same nature. He further conclude